In [3]:
import re
from pprint import pprint

In [73]:
path = './new_ontology.ttl'
base = 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#'
old_new = dict()
with open(path, 'r') as f:
    while line := f.readline():
        pars = re.search(r'(\S+) rdf:type (\S+) ;', line)
        if pars:
            if pars.group(2) != 'owl:Class': raise Exception('Unknown syntaxes')
            if pars.group(1).startswith(':'):
                old_new[pars.group(1)[1:]] = base + pars.group(1)[1:]

In [76]:
pprint(old_new)

{'Авиационная_и_космическая_психология': 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Авиационная_и_космическая_психология',
 'Авторское_право': 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Авторское_право',
 'Авторское_право_и_смежные_права': 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Авторское_право_и_смежные_права',
 'Авторское_право_и_смежные_права_в_отдельных_странах': 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Авторское_право_и_смежные_права_в_отдельных_странах',
 'Аграрное_право': 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Аграрное_право',
 'Аграрное_право_отдельных_стран': 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Аграрное_право_отдельных_стран',
 'Адвокатура': 'http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Адвокатура',
 'Административно-правовая_отве

In [79]:
new_file = []
with open(path, 'r') as f:
    while line := f.readline():
        pars_class = re.search(r'(\S+) rdf:type (\S+) ;', line)
        parse_subclass = re.search(r'rdfs:subClassOf :(\S+)', line[:-2])
        if pars_class:
            if pars_class.group(2) != 'owl:Class': raise Exception('Unknown syntaxes')
            if pars_class.group(1).startswith(':'):
                line = line.replace(pars_class.group(1), '<' + old_new[pars_class.group(1)[1:]] + '>')
        elif parse_subclass and parse_subclass.group(1) in old_new:
            line = line.replace(parse_subclass.group(1), '<' + old_new[parse_subclass.group(1)] + '>')
        new_file.append(line)
            

In [80]:
new_file

['@prefix : <http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19/> .\n', '@prefix owl: <http://www.w3.org/2002/07/owl#> .\n', '@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n', '@prefix xml: <http://www.w3.org/XML/1998/namespace> .\n', '@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n', '@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n', '@base <http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19/> .\n', '\n', '<http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19> rdf:type owl:Ontology .\n', '\n', '#################################################################\n', '#    Classes\n', '#################################################################\n', '\n', '###  http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Административное_производство_(административный_процесс)\n', '<http://www.semanticweb.org/григорий/ontologies/2024/10/untitled-ontology-19#Администра

In [84]:
with open('/home/viktor/dev/grant/onto2wiki-pyclient/fuc.ttl', 'w', encoding='utf-8') as f:
     f.write(''.join(new_file))

In [23]:
def parser_ttl(path):
    parent_children = dict()
    classes = dict()
    with open(path, 'r') as f:
        while line := f.readline():
            pars = re.search(r'(\S+) rdf:type (\S+) ;', line)
            if pars:
                if pars.group(2) != 'owl:Class': raise Exception('Unknown syntaxes')
                if pars.group(1).startswith(':'):
                    page = {'title': pars.group(1)[1:]}
                    while new_line := f.readline().strip():
                        pars_parent = re.search(r'rdfs:subClassOf :(\S+)', new_line[:-2])
                        if pars_parent:
                            pars_parent = pars_parent.group(1).split('#')
                            if len(pars_parent) == 2:
                                pars_parent = pars_parent[1][:-1]
                            else:
                                pars_parent = pars_parent[0]
                            
                            if pars_parent not in parent_children:
                                parent_children[pars_parent] = [pars.group(1)[1:],]
                            else:
                                parent_children[pars_parent].append(pars.group(1)[1:])
                        pars_label = re.search(r'rdfs:label "(.+)"@(\S+)', new_line[:-2])
                        if pars_label:
                            page.update({f'label@{pars_label.group(2)}': pars_label.group(1)})
                        if new_line[-1] == '.': break
                    classes[pars.group(1)[1:]] = page
                else:
                    pars = pars.group(1).split('#')[1][:-1]
                    page = {'title': pars}
                    
                    while new_line := f.readline().strip():
                        pars_parent = re.search(r'rdfs:subClassOf :(\S+)', new_line[:-2])
                        if pars_parent:
                            pars_parent = pars_parent.group(1).split('#')
                            if len(pars_parent) == 2:
                                pars_parent = pars_parent[1][:-1]
                            else:
                                pars_parent = pars_parent[0]
                            
                            if pars_parent not in parent_children:
                                parent_children[pars_parent] = [pars,]
                            else:
                                parent_children[pars_parent].append(pars)
                        pars_label = re.search(r'rdfs:label "(.+)"@(\S+)', new_line[:-2])
                        if pars_label:
                            page.update({f'label@{pars_label.group(2)}': pars_label.group(1)})
                        if new_line[-1] == '.': break
                    print(page)
                    break
                    classes[pars] = page
    # for parent, children in parent_children.items():
    #     classes[parent].update({'children': []})
    #     for child in children:
    #         classes[child]['parent'] = '_'.join(classes[parent]['title'].split())
    #         classes[parent]['children'].append('_'.join(classes[child]['title'].split()))
    # return classes


In [24]:
suka = (parser_ttl('new_ontology.ttl'))

{'title': 'Административное_производство_(административный_процесс)', 'label@ru': 'Административное производство (административный процесс)'}


In [7]:
from Onto2WikiClient import find_roots

In [11]:
find_roots(suka)

['Конституционное (государственное) право отдельных стран', 'Общие вопросы конституционного (государственного) права', 'Производство по делам о несостоятельности (банкротстве)', 'Производство по установлению фактов, имеющих юридическое значение', 'Участники процесса (субъекта)', 'Арбитражно-процессуальное право отдельных стран', 'Государственное устройство', 'Государственный суверенитет', 'Государство и право. Юридические науки', 'Гражданско-процессуальное право отдельных стран', 'Доказательства', 'Доказательства в арбитражном процессе', 'Законодательный процесс', 'Избирательное право. Избирательные системы. Выборы и референдумы', 'Иск', 'Исполнение судебных актов', 'Конституции и акты конституционного характера', 'Меры процессуального принуждения', 'Общие вопросы арбитражного процесса', 'Общие вопросы гражданского процесса', 'Общие вопросы уголовного процесса', 'Особенности производства по отдельным категориям гражданских дел', 'Особенности производства по отдельным категориям уголовн

In [12]:
suka['Конституционное (государственное) право отдельных стран']

KeyError: 'Конституционное (государственное) право отдельных стран'

In [14]:
pprint(suka)


{'Авиационная_и_космическая_психология': {'label@ru': 'Авиационная и '
                                                      'космическая психология',
                                          'parent': 'Прикладная_психология',
                                          'title': 'Авиационная_и_космическая_психология'},
 'Авторские_договоры': {'label@ru': 'Авторские договоры',
                        'parent': 'Авторское_право_и_смежные_права',
                        'title': 'Авторские_договоры'},
 'Авторское_право': {'label@ru': 'Авторское право',
                     'parent': 'Международное_частное_право',
                     'title': 'Авторское_право'},
 'Авторское_право_и_смежные_права': {'children': ['Авторские_договоры',
                                                  'Авторское_право_и_смежные_права_в_отдельных_странах',
                                                  'Имущественные_права_авторов_и_исполнителей',
                                                  'Использов